# Схемы разделения секрета

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime, getPrime
from Crypto.Random import random

# Утилитки
from functools import reduce

In [4]:
def prod(lst):
    """
    Произведение всех элементов списка.
    """
    
    return reduce((lambda x, y: x * y), lst)

----

In [5]:
class GenericParticipant(object):
    def __init__(self, x, y, t):
        """
        Инициализация участника разделения секрета.
        Сохраняются х, у и t.
        """
        
        self._x = x
        self._y = y
        self._t = t
        
        trace('[GenericParticipant]', f'{x = }, {y = }')
        self._verify()
        
        
    def __eq__(self, other):
        """
        Сравнение на равенство. Сравниваю по х - достаточно.
        """
        
        return self.x == other.x
        
        
    @property
    def x(self):
        """
        Получение х.
        """
        
        return self._x
    
    
    @property
    def y(self):
        """
        Получение у.
        """
        
        return self._y
        
        
    @property
    def equation(self):
        """
        Получение соответствующего участнику уравнения.
        """
        
        return [1] + [self._x ^ i for i in range(1, self._t)], self._y
    
    
    def _verify(self):
        pass

In [6]:
class GenericDealer(object):
    def __init__(self, t, n, field):
        """
        Инициализация дилера для разделения секрета.
        Генерация простого числа, поля и ключа.
        """
        
        self._field = field
        self._t = t
        self._n = n
        self._key = self._field.random_element()
        
        self._coefs = []
        self._values = []
        
        trace('[GenericDealer]', f't = {self._t}, n = {self._n}')
        trace('[GenericDealer]', self._field)
        trace('[GenericDealer]', f'key = {self._key}')
    
    
    @property
    def field(self):
        """
        Получение поля.
        """
        
        return self._field
    
    
    @property
    def key(self):
        """
        Получение ключа для проверки
        """
        
        return self._key
    
    
    def _prepare_coefficients(self):
        self._coefs  = [self._key] + [self._field.random_element() for _ in range(self._t - 1)]
        
        self._values = [sum([c * (x ^ i) for i, c in enumerate(self._coefs)]) 
                        for x in range(1, self._n + 1)]

----

In [7]:
def restore_secret_by_equation_system(field, participants: list[GenericParticipant]):
    """
    Восстановление секрета путем решения системы уравнений.
    """
    
    equations = [p.equation for p in participants]
    M = Matrix(field, [eq[0] for eq in equations])
    v = vector(field, (eq[1] for eq in equations))
    
    return M.solve_right(v)[0]  # Нас интересует только первая переменная,
                                # так как именно она является ключом

In [8]:
def restore_secret_by_lagrange_interpolation(participants: list[GenericParticipant]):
    """
    Восстановление секрета путем интерполяции Лагранжа.
    """
    
    return sum([p1.y * prod([p2.x * (p2.x - p1.x).inverse_of_unit() 
                             for p2 in participants if p1 != p2])
                for p1 in participants])

----

## Схема Шамира
![Shamir scheme](./images/Shamir_1_5.png)
![Shamir scheme](./images/Shamir_2_5.png)

In [9]:
class ShamirParticipant(GenericParticipant):
    def __init__(self, x, y, t):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirParticipant]', f'Initializing superclass...')
        super(ShamirParticipant, self).__init__(x, y, t)

In [10]:
class ShamirDealer(GenericDealer):
    def __init__(self, t, n):
        """
        Тут в целом все повторяет общий случай.
        """
        
        trace('[ShamirDealer]', f'Initializing superclass...')
        super(ShamirDealer, self).__init__(t, n, GF(getStrongPrime(512)))
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        return [ShamirParticipant(self._field.gen() + i, self._values[i], self._t) 
                for i in range(self._n)]

----

In [11]:
t, n = 3, 5

In [12]:
d = ShamirDealer(t, n)

[TRACE] [ShamirDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 10746683380914440290815134630835829077661721314628438796253279382516247354594062025443199219063084935050428325424313490762844028700058103432277887286683563
[TRACE] [GenericDealer] key = 10045238004899183111931167195097266430960383825377927813631063845723893110737028675787507264482217438584712350669038531071849760226391871691755126624300326


In [13]:
participants = d.split_key()

[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 8931095082307318365555266097244379408223821100821557335871023234941902571825940832977798974932782987051075722752505651248657184268706763819613304472273499
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 3029534368148659002943617905915264654093108358436581470713641593420150897634948604288319800971658451105119964729649957963074769047008777122833923270774160
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 3087239243337645314911357251945751246229966912851439014412198303674885442758114015162268961661928765797273402024784941977946543261356015033694870306485872
[TRACE] [ShamirParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 9104209707874277301458484135335839184634396764066129966966693365706106207195437065599646457003593931127536034637910603293272506911748477552196145579408635


In [14]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 10045238004899183111931167195097266430960383825377927813631063845723893110737028675787507264482217438584712350669038531071849760226391871691755126624300326
Restored correctly: True


In [15]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 7701542409337480995243886307330519201213541267435471770713869203650442942668522168420192364002807906133132220245702803395721059882327180125749588248823248
Restored correctly: False


In [16]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 10045238004899183111931167195097266430960383825377927813631063845723893110737028675787507264482217438584712350669038531071849760226391871691755126624300326
Restored correctly: True


In [17]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 520415721987289070890265667116177604122316240692563689617857657540801239748665589600536037385272779514653525086218583202136300986307861888793249674614480
Restored correctly: False


---
## Схема Фельдмана
![Feldman scheme](./images/Feldman_5.png)

In [18]:
class FeldmanParticipant(GenericParticipant):
    def __init__(self, x, y, t, g, check_values):
        """
        Тут в дополнение сохраняется генератор и проверочные значения.
        """
        
        self._g = g
        self._check_values = check_values
        
        trace('[FeldmanParticipant]', f'Initializing superclass...')
        super(FeldmanParticipant, self).__init__(x, y, t)
        
        
    def _verify(self):
        expected_value = self._g ^ self.y
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with x = {self.x} and y = {self.y}')
            
        trace('[FeldmanParticipant]', 'Value verified')

In [19]:
class FeldmanDealer(GenericDealer):
    def __init__(self, t, n):
        """
        В дополнение генерируются p, q, g.
        """
        
        self._p, self._q, self._g = FeldmanDealer._generate_parameters()
        
        trace('[FeldmanDealer]', f'Initializing superclass...')
        super(FeldmanDealer, self).__init__(t, n, GF(self._q))
        
        trace('[FeldmanDealer]', f'p = {self._p}')
        trace('[FeldmanDealer]', f'q = {self._q}')
        trace('[FeldmanDealer]', f'g = {self._g}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        check_values = [self._g ^ c for c in self._coefs]
        
        return [FeldmanParticipant(self._field.gen() + i, self._values[i], self._t, self._g, check_values) 
                for i in range(self._n)]
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        return p, q, _generate_special_group_element(p, q)

----

In [20]:
d = FeldmanDealer(t, n)

[TRACE] [FeldmanDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 859000945459477133793765991761850874983659080821
[TRACE] [GenericDealer] key = 54293937948937588760047366379474829061523845870
[TRACE] [FeldmanDealer] p = 71615560485743098112088464729310737852955203639395099083230484298810368257766922298246091455969201689838655477480383411149351653675885489277145173073696987084332841964773028240986897267025206226294921382491672522707710637765801027983613459461761676285311516678136126633087743879657591947994424506037058993239
[TRACE] [FeldmanDealer] q = 859000945459477133793765991761850874983659080821
[TRACE] [FeldmanDealer] g = 6748591219838678756998731377228509433576681605735132994651833523184225156480809995315398526170682541881129028068994369320735732249623320857740104611665597989164239535071616298434212253844024536882846574166144394155741914864371709620337274068942564569242515143932053359644580132921402756572765611614

In [21]:
participants = d.split_key()

[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 559036510485148874041077908139407749827876333790
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 751249327205626652217850008480573505182075147913
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 630932388110370923290363667402972095124120288239
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 198085693199381687258618884906603519654011754768
[TRACE] [FeldmanParticipant] Value verified
[TRACE] [FeldmanParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 311710187932136077916381652753318653755408628321
[TRACE] [FeldmanParticipant] Value verified


In [22]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 54293937948937588760047366379474829061523845870
Restored correctly: True


In [23]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 93588098942799282519552032626700139687924816155
Restored correctly: False


In [24]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 54293937948937588760047366379474829061523845870
Restored correctly: True


In [25]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 93588098942799282519552032626700139687924816155
Restored correctly: False


----
## Схема Педерсена
![Pedersen scheme](./images/Pedersen_5.png)

In [26]:
class PedersenParticipant(GenericParticipant):
    def __init__(self, i, u, w, g, h, t, check_values):
        """
        Инициализация участника разделения секрета.
        Сохраняются проверочные значения и элементы группы GF*(p)
        """
        
        self._g, self._h = g, h
        self._w = w
        self._check_values = check_values
        
        trace('[PedersenParticipant]', f'Initializing superclass...')
        super(PedersenParticipant, self).__init__(i, u, t)
        
    
    def _verify(self):
        expected_value = (self._g ^ self.y) * (self._h ^ self._w)
        value_to_check = prod([cv ^ (self.x ^ i) for i, cv in enumerate(self._check_values)])
        
        if value_to_check != expected_value:
            raise ValueError(f'Wrong value given to the participant with i = {self.x}')
            
        trace('[PedersenParticipant]', 'Value verified')

In [27]:
class PedersenDealer(GenericDealer):
    def __init__(self, t, n):
        """
        В дополнение к общему случаю генерируются 4 параметра: p, q, g и h.
        """
        
        self._p, self._q, self._g, self._h = PedersenDealer._generate_parameters()
        self._gammas = []
        
        trace('[PedersenDealer]', f'Initializing superclass...')
        super(PedersenDealer, self).__init__(t, n, GF(self._q))
        
        trace('[PedersenDealer]', f'p = {self._p}')
        trace('[PedersenDealer]', f'q = {self._q}')
        trace('[PedersenDealer]', f'g = {self._g}')
        trace('[PedersenDealer]', f'h = {self._h}')
        
        
    def split_key(self):
        """
        Разделение секрета между участниками.
        """
        
        self._prepare_coefficients()
        self._gammas = [self._field.random_element() for _ in range(t)]
        
        check_values = [(self._g ^ d) * (self._h ^ g)
                        for d, g in zip(self._coefs, self._gammas)]
        
        return [PedersenParticipant(self._field.gen() + i, u, self._w(i + 1),
                                    self._g, self._h, self._t, check_values) 
                for i, u in enumerate(self._values)]
    
    
    def _w(self, z):
        return sum([g * (z ^ i) for i, g in enumerate(self._gammas)])
    
    
    @staticmethod
    def _generate_parameters():
        def _generate_special_group_element(p, q):
            #
            # Сгенерирую число, взаимно простое с p
            #

            tmp = random.getrandbits(P_LENGTH)
            while gcd(tmp, p) != 1:
                tmp = random.getrandbits(P_LENGTH)

            #
            # По малой теореме Ферма такой g будет иметь порядок q
            #

            F = GF(p)
            return F(tmp) ^ ((p - 1) // q)
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = getPrime(Q_LENGTH)
        p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
        
        while p not in Primes():
            p = random.getrandbits(P_LENGTH - Q_LENGTH) * q + 1
            
        g = _generate_special_group_element(p, q)
        h = _generate_special_group_element(p, q)
        
        while g == h:
            h = _generate_special_group_element(p, q)
            
        return p, q, g, h

-----

In [28]:
d = PedersenDealer(t, n)

[TRACE] [PedersenDealer] Initializing superclass...
[TRACE] [GenericDealer] t = 3, n = 5
[TRACE] [GenericDealer] Finite Field of size 1191888728202286622891746897673895367840511496593
[TRACE] [GenericDealer] key = 2153320293527776387604677503690549982684444819
[TRACE] [PedersenDealer] p = 73783971987918543262012390416364351593722708804975899503308334857385539182699612665250576455069975842108145445936446427875601460086060739421287610752804991316361069982701736886660564677571804121887655954541409744949537333872478682651335427543903167361108895413085196269897308530173292155306641077148131876531
[TRACE] [PedersenDealer] q = 1191888728202286622891746897673895367840511496593
[TRACE] [PedersenDealer] g = 54139404434865636396785307121606398202323517962982765620154143872090546551655598834382664070414854586620668017549565777316175282773019296027811639792913744135505917934389393638429218277796656268790383159344960013265895043895297569929277255785160743121312025959331315309466312522647525411585604

In [29]:
participants = d.split_key()

[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 1, y = 84192018786482499725372344552002180040237097031
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 2, y = 595091814381710945492911590002254697391378022800
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 3, y = 342963978876926490798475516180552734195595725533
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 4, y = 519697240474415758533811020760791658293401701823
[TRACE] [PedersenParticipant] Value verified
[TRACE] [PedersenParticipant] Initializing superclass...
[TRACE] [GenericParticipant] x = 5, y = 1125291599174178748698918103742971469684795951670
[TRACE] [PedersenParticipant] Value verified


In [30]:
restored_key = restore_secret_by_equation_system(d.field, 
                                                 random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 2153320293527776387604677503690549982684444819
Restored correctly: True


In [31]:
false_key = restore_secret_by_equation_system(d.field, 
                                              random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 717833669841273404650796078009681383509481005816
Restored correctly: False


In [32]:
restored_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t))
print(f'Restored key = {restored_key}')
print(f'Restored correctly: {restored_key == d.key}')

Restored key = 2153320293527776387604677503690549982684444819
Restored correctly: True


In [33]:
false_key = restore_secret_by_lagrange_interpolation(random.sample(participants, t - 1))
print(f'Restored key = {false_key}')
print(f'Restored correctly: {false_key == d.key}')

Restored key = 765180951393540676849579996775645030529607667855
Restored correctly: False
